# Load Teslafi CSV with SCHEMA to MySQL/MariaDB or Spark SQL using PySpark

It is very useful in the data exploration or descriptive analytics phase of a project to be able to query your CSV files more or less directly using the power of SQL.

Spark makes this very simple by creating tables in Hive that reference the CSVs. In addition to SQL, this gives us the additional capabilities of the Spark and Pandas data frames.

Spark then allows us to create permanent tables in Hive using the very efficient Parquet file format. 

If we then want to store to a database outside of the Spark environment, we can save those dataframes to MySQL/MariaDB or other JDBC compliant databases.

## Prep the Spark server with the required JDBC driver `.jar` file

On Databricks, this can be done in the UI: Compute -> cluster -> Libraries

## Prep the target MySQL/MariaDB server
Create the database you want to use beforehand.  This code will create tables, but not the database.

On local Spark, the `.jar` can be placed in `$SPARK_HOME/jars/`

## Import libraries and configure secrets

In this example, we are using the `configparser` library to read a simple `.ini` style file named `CREDENTIALS.config`.  If you choose to use this method, create a section in the file like this:
```
[csvload]
sqluser = <myuser>
sqlpassword = <mypass>
sqlhost = <host or ip>
sqlport = <3306 or custom port>
sqldb = <database name>
```


In [ ]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, TimestampType, LongType, DoubleType
from pyspark.sql.types import DateType

import pandas as pd
from configparser import ConfigParser

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/share/java/mariadb-java-client.jar pyspark-shell'

import findspark
findspark.init()


In [ ]:
# !env|grep SPARK
# !cp -v /usr/share/java/mariadb-java-client.jar $SPARK_HOME/jars/

# Establish our Spark session

In [ ]:
# spark = SparkSession.builder.config("spark.jars", "/usr/share/java/mariadb-java-client.jar").appName("TeslafiLoad").getOrCreate()
spark = SparkSession.builder.appName("CSVLoad").getOrCreate()
sc = spark.sparkContext

In [ ]:
# !echo $VIRTUAL_ENV
# !env|grep SPARK
!echo $SPARK_HOME
# !cp -v  /usr/share/java/mariadb-java-client.jar $SPARK_HOME/jars/
# !ls $SPARK_HOME/jars
# # !pip install pandas findspark

In [ ]:
# os.listdir(path='/data/data-files/teslafi/')

# Optional - create a Spark/Hive temporary table from the data

# Write CSVs to MySQL/MariaDB


## Set all the parameters
For privacy, you may prefer to load `csvdir` from `CREDENTIALS.config`.  

The same goes for all the parameters.  Default practice is load everything from the credentials file, 
then override one or two variables as long as the values would not cause problems if checked in to a public repo.


In [ ]:
# File location and type

file_type = "csv"
# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# MySQL/MariaDB info
# Get credentials from file
config_section = 'local'
parser = ConfigParser()
_ = parser.read('CREDENTIALS.config')

# csvdir = parser.get(config_section, 'csvdir')
# csvdir = '/home/gregj/projects/private-orgmode/stardog/engagements/vizient/from_customer/'
csvdir = '/data/data-files/teslafi/data/'
csvdir = '/data/data-files/teslafi/'  ## CHANGE ME BACK!!!!
user = parser.get(config_section, 'sqluser')
password = parser.get(config_section, 'sqlpassword')
host = parser.get(config_section, 'sqlhost')
port = parser.get(config_section, 'sqlport')
# db = parser.get(config_section, 'sqldb')
db = 'teslafi'
sqlurl = 'jdbc:mysql://' + host + ':' + port + '/' + db

# csvdir = '/data/graph-data/AntiFraud/data/'
# csvdir = '/data/data-files/teslafi/'
# csvdir = '/data/graph-data/ldbc-workshop/data/'

In [ ]:
teslafi_schema = StructType(
    [
        StructField('data_id', IntegerType(), False),
        StructField('Date', TimestampType(), False),
        StructField('calendar_enabled', IntegerType(), True),
        StructField('remote_start_enabled', IntegerType(), True),
        StructField('vehicle_id', IntegerType(), True),
        StructField('display_name', StringType(), True),
        StructField('color', StringType(), True),
        StructField('backseat_token', StringType(), True),
        StructField('notifications_enabled', IntegerType(), True),
        StructField('vin', StringType(), True),
        StructField('backseat_token_updated_at', StringType(), True),
        StructField('id', LongType(), True),
        StructField('tokens', StringType(), True),
        StructField('id_s', StringType(), True),
        StructField('state', StringType(), True),
        StructField('user_charge_enable_request', StringType(), True),
        StructField('time_to_full_charge', DoubleType(), True),
        StructField('charge_current_request', IntegerType(), True),
        StructField('charge_enable_request', IntegerType(), True),
        StructField('charge_to_max_range', IntegerType(), True),
        StructField('charger_phases', StringType(), True),
        StructField('battery_heater_on', StringType(), True),
        StructField('managed_charging_start_time', StringType(), True),
        StructField('battery_range', DoubleType(), True),
        StructField('charger_power', IntegerType(), True),
        StructField('charge_limit_soc', IntegerType(), True),
        StructField('charger_pilot_current', IntegerType(), True),
        StructField('charge_port_latch', StringType(), True),
        StructField('battery_current', StringType(), True),
        StructField('charger_actual_current', IntegerType(), True),
        StructField('scheduled_charging_pending', IntegerType(), True),
        StructField('fast_charger_type', StringType(), True),
        StructField('usable_battery_level', IntegerType(), True),
        StructField('motorized_charge_port', IntegerType(), True),
        StructField('charge_limit_soc_std', IntegerType(), True),
        StructField('not_enough_power_to_heat', StringType(), True),
        StructField('battery_level', IntegerType(), True),
        StructField('charge_energy_added', DoubleType(), True),
        StructField('charge_port_door_open', StringType(), True),
        StructField('max_range_charge_counter', IntegerType(), True),
        StructField('charge_limit_soc_max', IntegerType(), True),
        StructField('ideal_battery_range', DoubleType(), True),
        StructField('managed_charging_active', IntegerType(), True),
        StructField('charging_state', StringType(), True),
        StructField('fast_charger_present', IntegerType(), True), # can be None
        StructField('trip_charging', IntegerType(), True), # None
        StructField('managed_charging_user_canceled', IntegerType(), True),
        StructField('scheduled_charging_start_time', LongType(), True),
        StructField('est_battery_range', DoubleType(), True),
        StructField('charge_rate', DoubleType(), True),
        StructField('charger_voltage', IntegerType(), True),
        StructField('charge_current_request_max', IntegerType(), True),
        StructField('eu_vehicle', IntegerType(), True),
        StructField('charge_miles_added_ideal', DoubleType(), True),
        StructField('charge_limit_soc_min', IntegerType(), True),
        StructField('charge_miles_added_rated', DoubleType(), True),
        StructField('inside_temp', DoubleType(), True),
        StructField('longitude', DoubleType(), True),
        StructField('heading', IntegerType(), True),
        StructField('gps_as_of', IntegerType(), True),
        StructField('latitude', DoubleType(), True),
        StructField('speed', IntegerType(), True),
        StructField('shift_state', StringType(), True),
        StructField('seat_heater_rear_right', StringType(), True),
        StructField('seat_heater_rear_left_back', StringType(), True),
        StructField('seat_heater_left', StringType(), True),
        StructField('passenger_temp_setting', DoubleType(), True),
        StructField('is_auto_conditioning_on', IntegerType(), True), # None
        StructField('driver_temp_setting', DoubleType(), True),
        StructField('outside_temp', DoubleType(), True),
        StructField('seat_heater_rear_center', StringType(), True),
        StructField('is_rear_defroster_on', IntegerType(), True),
        StructField('seat_heater_rear_right_back', StringType(), True),
        StructField('smart_preconditioning', StringType(), True),
        StructField('seat_heater_right', StringType(), True),
        StructField('fan_status', IntegerType(), True),
        StructField('is_front_defroster_on', IntegerType(), True), # None
        StructField('seat_heater_rear_left', StringType(), True),
        StructField('gui_charge_rate_units', StringType(), True),
        StructField('gui_24_hour_time', StringType(), True),
        StructField('gui_temperature_units', StringType(), True),
        StructField('gui_range_display', StringType(), True),
        StructField('gui_distance_units', StringType(), True),
        StructField('sun_roof_installed', IntegerType(), True),
        StructField('rhd', IntegerType(), True),
        StructField('remote_start_supported', IntegerType(), True),
        StructField('homelink_nearby', IntegerType(), True),
        StructField('parsed_calendar_supported', IntegerType(), True),
        StructField('spoiler_type', StringType(), True),
        StructField('ft', IntegerType(), True),
        StructField('odometer', DoubleType(), True),
        StructField('remote_start', IntegerType(), True), # None
        StructField('pr', IntegerType(), True),
        StructField('has_spoiler', StringType(), True),
        StructField('roof_color', StringType(), True),
        StructField('perf_config', StringType(), True),
        StructField('valet_mode', IntegerType(), True),
        StructField('calendar_supported', IntegerType(), True),
        StructField('pf', IntegerType(), True),
        StructField('sun_roof_percent_open', IntegerType(), True),
        StructField('third_row_seats', StringType(), True),
        StructField('seat_type', IntegerType(), True),
        StructField('api_version', IntegerType(), True),
        StructField('rear_seat_heaters', IntegerType(), True),
        StructField('rt', IntegerType(), True),
        StructField('exterior_color', StringType(), True),
        StructField('df', IntegerType(), True),
        StructField('autopark_state', StringType(), True),
        StructField('sun_roof_state', StringType(), True),
        StructField('notifications_supported', IntegerType(), True),
        StructField('vehicle_name', StringType(), True),
        StructField('dr', IntegerType(), True),
        StructField('autopark_style', StringType(), True),
        StructField('car_type', StringType(), True),
        StructField('wheel_type', StringType(), True),
        StructField('locked', IntegerType(), True), # None
        StructField('center_display_state', IntegerType(), True),
        StructField('last_autopark_error', StringType(), True),
        StructField('car_version', StringType(), True),
        StructField('dark_rims', IntegerType(), True),
        StructField('autopark_state_v2', StringType(), True),
        StructField('inside_tempF', StringType(), True),
        StructField('driver_temp_settingF', StringType(), True),
        StructField('outside_tempF', StringType(), True),
        StructField('odometerF', StringType(), True),
        StructField('idleNumber', IntegerType(), True),
        StructField('sleepNumber', IntegerType(), True),
        StructField('driveNumber', IntegerType(), True),
        StructField('chargeNumber', IntegerType(), True),
        StructField('polling', StringType(), True),
        StructField('idleTime', IntegerType(), True),
        StructField('running', IntegerType(), True),
        StructField('rerunning', IntegerType(), True),
        StructField('maxRange', DoubleType(), True),
        StructField('left_temp_direction', IntegerType(), True),
        StructField('max_avail_temp', DoubleType(), True),
        StructField('is_climate_on', IntegerType(), True),
        StructField('right_temp_direction', IntegerType(), True),
        StructField('min_avail_temp', DoubleType(), True),
        StructField('rear_seat_type', IntegerType(), True),
        StructField('power', IntegerType(), True),
        StructField('steering_wheel_heater', IntegerType(), True),
        StructField('wiper_blade_heater', IntegerType(), True),
        StructField('side_mirror_heaters', IntegerType(), True),
        StructField('elevation', DoubleType(), True)
    ]
)

In [ ]:
os.listdir(path=csvdir)

## Think about creating the table first!

File is at /data/data-files/teslafi/mysql-create-table.sql

This will create the table with indices!


Then use 'append' or 'overwrite' below

## Write the CSVs to the database
Database must be created beforehand, but not the tables

Modify the rules paying special attention to the `timestamp` columns in the dataframe.  `TIMESTAMP` column type in MySQL cannot represent dates prior to 1980.  If needed, transfer those dataframe columes to `date` prior to creating the MySQL table with `df.write()`

In [ ]:
# # for path,name in [(f.path,f.name) for f in dbutils.fs.ls(csvdir) if f.path.endswith('.csv')  ]:
for fname in os.listdir(path=csvdir):
    # if fname.endswith('.csv') and fname.startswith('teslafi-20'):
    if fname.startswith('charges.csv'):
        print(fname)
        # if not fname in ['ff_party.csv', 'ff_company.csv']:
        #     print('skipping ' + fname)
        #     continue
        # .schema(teslafi_schema) \  ## CHANGE ME BACK
        df = spark.read.format('csv') \
        .option("inferSchema", True) \
        .option("header", first_row_is_header) \
        .option("sep", delimiter) \
        .option("na_values", ["None", None]) \
        .load(csvdir + '/' + fname)
        # timestamp type in MySQL represents dates only after 1980
        # if timestamp is desired, comment or modify the following code
        # this code changes *all* timestamp columns to Date type
        # for a,b in df.dtypes:
        #     if b == "timestamp" and a.find("timestamp") < 0: 
        #         newdf = df.withColumn(a, df[a].cast(DateType()))
        #         print('   CHANGED column ' + a + ' to Date type')
        #         df = newdf
        #     if '.' in a:
        #         df.withColumnRenamed(a, a.replace(',', '_'))
        # Modify filename-to-tablename as needed
        # break
        df.write.format('jdbc').options(
            url=sqlurl,
            driver='org.mariadb.jdbc.Driver',
            # dbtable=fname.replace('.csv','').replace('-', '_').replace('c360_','').replace('ce_','').replace('exampledata',''),
            # dbtable='teslafi',
            dbtable='charges',   ### CHANGE ME BACK
            user=user,
            password=password).mode('append').save()        
        # append or overwrite or ignore

In [ ]:
# pprint
df.printSchema()

In [ ]:
df